# Create Layer for Lambda

## Version 1 Native

see <https://nordcloud.com/lambda-layers-for-python-runtime/>

In [ ]:
!rm -rf build
PY_DIR='build/python/lib/python3.7/site-packages'
!mkdir -p {PY_DIR}
!pip install -r requirements.txt --no-deps -t {PY_DIR}

Change Permission

In [ ]:
!chmod -R go+r build/
!find build/ -type d -exec chmod 755 {} \;

Zip 

In [ ]:
!rm sklearn.zip
!cd {PY_DIR} && zip -r9 ../../../../../sklearn.zip .   

## Version 2 Docker

<https://medium.com/@qtangs/creating-new-aws-lambda-layer-for-python-pandas-library-348b126e9f3e>

In [ ]:
import os
PKG_DIR="python"
CWD = os.getcwd()

In [ ]:
!rm -rf {PKG_DIR} && mkdir -p {PKG_DIR}

!docker run --rm -v {CWD}:/foo -w /foo lambci/lambda:build-python3.7 \
    pip install -r requirements.txt --no-deps -t {PKG_DIR}

In [ ]:
PKG_DIR="python"
!chmod -R go+r {PKG_DIR}
!find {PKG_DIR} -type d -exec chmod 755 {} \;

In [ ]:
!rm sklearn.zip
!zip -r sklearn.zip .

# Upload to AWS

In [ ]:
!aws s3 mb s3://lambda-scikit-bucket --region us-east-1 --profile dev

In [ ]:
!aws s3 cp ./sklearn.zip s3://lambda-scikit-bucket/ --profile dev

In [ ]:
!aws s3 ls s3://lambda-scikit-bucket/ --profile dev

In [ ]:
!aws lambda publish-layer-version \
                        --region us-east-1 --layer-name sklearn \
                        --compatible-runtimes python3.7\
                        --content S3Bucket=lambda-scikit-bucket,S3Key=sklearn.zip --profile dev

In [ ]:
!aws s3 cp s3://lambda-scikit-bucket/sklearn.zip . --profile dev

# Delete all Layer Buckets

In [ ]:
import os
output=!aws s3 ls --profile dev | awk -F" " '{print $3}'
for i in output:
    print(i)
    os.system("aws s3 rm s3://%s --recursive --profile dev"%i)
    os.system("aws s3 rb s3://%s --profile dev"%i)